Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils

import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files = [ 'Quake_10x_Spleen',
    'Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',

 'Young',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 '10X_PBMC'
        ]

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [4]:
optimal = pd.read_pickle(f"{path}output/pickle_results/real_data/optimal_input_size.pkl")
optimal.head()

,dataset,nb_genes
0,Quake_Smart-seq2_Trachea,500
1,Quake_Smart-seq2_Diaphragm,1000
2,Quake_10x_Spleen,500
3,Young,1000
4,mouse_ES_cell,1000


In [5]:
# df = pd.read_pickle(f"{path}output/pickle_results/real_data/real_data_layers.pkl")
df = pd.read_pickle(f"{path}output/pickle_results/real_data/real_data_layers_new.pkl")

In [15]:
# df = pd.DataFrame()
lr = 0.4
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])
    nb_genes = optimal[optimal["dataset"] == dataset]["nb_genes"].values[0]

    for temperature in [0.07]:
        for dropout in [0.9,
#                         0.7, 0.2, 0.5 
                       ]:

            for layers in [ 
#                 [200, 40, 2], [10],[50, 10], [200, 40, 10], [200, 40, 20, 10],
#                 [2], [20], [40], [60], [100],
#                 [50, 2], [50, 20], [50, 40], [80, 60], [100, 100],
#                 [200, 40, 80],  [200, 20, 60],[200, 40, 60, 40], [200, 40, 60, 60],
#                 [200, 100, 100], [200, 100, 60], [200, 40, 40], [200, 40, 60],
#                 [100, 60, 40],
#                 [200, 100, 60, 40], [200, 60, 40, 20], [100, 80, 60, 50],
                [300, 40, 60], [300, 60, 60], [300, 40, 40],

                
#                     [200, 60, 40, 2],[200, 60, 2], [60, 2], [2],
#                     [200, 60, 60]
#                     [200, 100, 100], [200, 40, 100], [200, 40, 40], [200, 40, 60]
#                     [200, 100, 100, 100],[200, 80, 100], [100, 100], [100],
#                     [200, 100, 80, 80],[200, 80, 80], [100, 80], [80],
#                     [200, 100, 60, 60],[200, 80, 60], [100, 60], [60],
#                     [200, 60, 40, 40],[200, 60, 40], [60, 40], [40],
#                     [200, 60, 40, 20],[200, 60, 20], [60, 20], [20],
                          ]:
                data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
                X = np.array(data_mat['X'])
                Y = np.array(data_mat['Y'])
                cluster_number = np.unique(Y).shape[0]

                X = train.preprocess(X, nb_genes=nb_genes)

                for run in range(3):
                    torch.manual_seed(run)
                    torch.cuda.manual_seed_all(run)
                    np.random.seed(run)
                    random.seed(run)
                    torch.backends.cudnn.deterministic = True
                    torch.backends.cudnn.benchmark = False
                    dresults = train.run(X,
                                         cluster_number,
                                         dataset,
                                         Y=Y,
                                         nb_epochs=30,
                                         lr=lr,
                                         temperature=temperature,
                                         dropout=dropout,
                                         
                                         layers=layers,
                                         save_to=f"{path}output/real_data/{run}/",
                                         save_pred = False)
                    dresults["nb_genes"] = nb_genes
                    dresults["dropout"] = dropout
                    dresults["lr"] = lr
                    dresults["layers"] = str(layers)
                    dresults["run"] = run
                    print(f".", end = "")
                    df = df.append(dresults, ignore_index=True)

                    df.to_pickle(f"{path}output/pickle_results/real_data/real_data_layers_new.pkl")

>>>>> Data Quake_10x_Spleen
SCZI  0.9207243162607069
(9552, 23341) (9552, 23341) keeping 500 genes
...(9552, 23341) (9552, 23341) keeping 500 genes
...(9552, 23341) (9552, 23341) keeping 500 genes
...>>>>> Data Quake_Smart-seq2_Trachea
SCZI  0.8258284521205065
(1350, 23341) (1350, 23341) keeping 500 genes
...(1350, 23341) (1350, 23341) keeping 500 genes
...(1350, 23341) (1350, 23341) keeping 500 genes
...>>>>> Data Quake_Smart-seq2_Diaphragm
SCZI  0.9596701189611787
(870, 23341) (870, 23341) keeping 1000 genes
...(870, 23341) (870, 23341) keeping 1000 genes
...(870, 23341) (870, 23341) keeping 1000 genes
...>>>>> Data Young
SCZI  0.6966379900869195
(5685, 33658) (5685, 33658) keeping 1000 genes
...(5685, 33658) (5685, 33658) keeping 1000 genes
...(5685, 33658) (5685, 33658) keeping 1000 genes
...>>>>> Data mouse_ES_cell
SCZI  0.8004502714264402
(2717, 24175) (2717, 24175) keeping 1000 genes
...(2717, 24175) (2717, 24175) keeping 1000 genes
...(2717, 24175) (2717, 24175) keeping 1000 ge

In [25]:
df.layers.unique().shape

(30,)

In [14]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")
sczi = sczi[sczi["dataset"].isin(files)]
sczi.mean()

ARI       0.752219
NMI       0.780598
sil       0.256274
run       1.000000
time    189.880156
cal     596.273096
dtype: float64

In [20]:
# df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_layers.pkl")
df = df[df["dataset"].isin(files)]
r = df.groupby([ "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "lr"]).T
r

,dropout,0.9
layers,lr,
"[100, 100]",0.4,0.769378
"[100, 60, 40]",0.4,0.763490
"[100, 80, 60, 50]",0.4,0.766125
[100],0.4,0.731138
[10],0.4,0.749402
"[200, 100, 100]",0.4,0.756084
"[200, 100, 60, 40]",0.4,0.751667
"[200, 100, 60]",0.4,0.763108
"[200, 20, 60]",0.4,0.762309


In [10]:
# df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_layers.pkl")
df = df[df["dataset"].isin(files)]
r = df.groupby([ "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "lr"]).T
r

,dropout,0.9
layers,lr,
"[100, 100]",0.4,0.769378
[100],0.4,0.731138
[20],0.4,0.758565
[2],0.4,0.540770
[40],0.4,0.746133
"[50, 20]",0.4,0.758853
"[50, 2]",0.4,0.465332
"[50, 40]",0.4,0.753225
[60],0.4,0.732374


In [ ]:
r.mean(axis = 1)

In [ ]:
# df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "lr"]).max()

In [ ]:
# r = df[(df["layers"]=="[40]")
#        &(df["temperature"]==0.01)
#        &(df["lr"]==1e-5)
#        &(df["dropout"]==0.9)
#       ]
r = df[(df["layers"]=="[200, 100, 30, 30]")
       &(df["temperature"]==0.07)
       &(df["lr"]==1e-5)
       &(df["dropout"]==0.9)
      ]


r.mean()

In [ ]:
comb = pd.merge(r, sczi, on=["dataset", "run"])[[
    "dataset", "kmeans_ari_0", "ARI", "kmeans_nmi_0", "NMI"
]].rename(columns = {"kmeans_ari_0": "contrative-sc", "ARI": "sczi"})
comb

In [ ]:
comb =pd.melt(comb, id_vars=["dataset"], value_vars=["contrative-sc", "sczi"])

In [ ]:
comb.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot( data = comb, y="value", x = "variable")
plt.xticks(rotation = 90)

In [ ]:
sns.barplot(x = "dataset", data = comb, y="value", hue = "variable")
plt.xticks(rotation = 90)

In [ ]:
df.groupby("dataset").mean()

# Importance of input size

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_genes", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    for run in range(2):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        for nb_genes in [100, 200, 500, 1000, 1500, 2000, 5000, 8000]:

            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
            print(f"Perc 0 {perc_0}")
            cluster_number = np.unique(Y).shape[0]

            X = train.preprocess(X, nb_genes=nb_genes)
            nb_zeros = int(0.8 * nb_genes)
            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                
                epochs=100,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/inputs/{dataset}_{nb_genes}/")

            #         df.loc[df.shape[0]] = [
            #                 dataset, perc_0, nb_genes, 'kmeans_representation_0',dresults['kmeans_representation_0']
            #             ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, nb_genes, 'leiden_representation_0',
                dresults['leiden_representation_0'], run]

#             pxt = PCA(2).fit_transform(X)
#             dresults["original"] = utils.evaluate(X, Y, cluster_number)[1]
#             dresults["pca"] = utils.evaluate(pxt, Y, cluster_number)[1]
            print(dresults)
    df.to_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df.groupby(["nb_genes"]).mean()

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_genes": "Nb input genes"})

In [ ]:
df["dataset"].unique()


In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size.pdf", bbox_inches='tight')

In [ ]:
datasets = ['10X PBMC (2100)',
       'Mouse ES\nCell (2100)', 'Worm Neuron\nCell (2100)',
       'Mouse Bladder\n Cell (2100)']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size_2100.pdf", bbox_inches='tight')

# Importance of the number of epochs

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_epochs", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    nb_genes = 1500
    for epochs in [5, 50, 100, 300]:

        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
        print(f"Perc 0 {perc_0}")
        cluster_number = np.unique(Y).shape[0]

        X = train.preprocess(X, nb_genes=nb_genes)
        nb_zeros = int(0.8 * nb_genes)
        for run in range(2):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                
                epochs=epochs,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/epochs/{dataset}_{epochs}/")

            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'kmeans_representation_0',
                dresults['kmeans_representation_0'], run
            ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'leiden_representation_0',
                dresults['leiden_representation_0'], run
            ]

            print(dresults)
            df.to_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_epochs": "Nb epochs"})

In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(7, 3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb epochs",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title ="Number of epochs",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_nb_epochs.pdf", bbox_inches='tight')